<a href="https://colab.research.google.com/github/adrianacupp/fake_new_classification/blob/main/Modeling_merged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing data and models

In [3]:
import pandas as pd

In [4]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import pandas as pd 
df =pd.read_csv('gdrive/My Drive/Colab Notebooks/FAKE_NEWS/df_pre_tok_text.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71425 entries, 0 to 71424
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  71425 non-null  int64 
 1   text        71425 non-null  object
 2   label       71425 non-null  int64 
 3   text_clean  71425 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.2+ MB


In [7]:
df.dropna(axis=0, how='any', inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71425 entries, 0 to 71424
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  71425 non-null  int64 
 1   text        71425 non-null  object
 2   label       71425 non-null  int64 
 3   text_clean  71425 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.7+ MB


In [9]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 41.0 MB/s 
     |████████████████████████████████| 6.6 MB 64.4 MB/s 
     |████████████████████████████████| 120 kB 75.6 MB/s 


In [10]:
import torch
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [11]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.3590, 0.2632, 0.8644],
        [0.7309, 0.9407, 0.0552],
        [0.7752, 0.7871, 0.7013],
        [0.2396, 0.4766, 0.8467],
        [0.6967, 0.3909, 0.6622]])


In [12]:
config = BertConfig.from_pretrained('bert-base-uncased',finetuning_task='binary')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [14]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [15]:
def get_tokens(text, tokenizer, max_seq_length, add_special_tokens=True): 
    input_ids = tokenizer.encode(text, 
                                 add_special_tokens=add_special_tokens, 
                                 max_length=max_seq_length, 
                                 padding='max_length',
                                 truncation=True) 
    attention_mask = [int(id > 0) for id in input_ids] 
    assert len(input_ids) == max_seq_length 
    assert len(attention_mask) == max_seq_length 
    return (input_ids, attention_mask) 
text = "Here is the sentence I want embeddings for." 
input_ids, attention_mask = get_tokens(text, 
                                       tokenizer, 
                                       max_seq_length=30,
                                       add_special_tokens = True) 
input_tokens = tokenizer.convert_ids_to_tokens(input_ids) 
print (text) 
print (input_tokens) 
print (input_ids) 
print (attention_mask)

Here is the sentence I want embeddings for.
['[CLS]', 'here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[101, 2182, 2003, 1996, 6251, 1045, 2215, 7861, 8270, 4667, 2015, 2005, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


#Modeling

##Train set

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
df

,Unnamed: 0,text,label,text_clean
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1,law enforcement on high alert following threat...
1,1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1,unbelievable obama s attorney general says mos...
2,2,"Bobby Jindal, raised Hindu, uses story of Chri...",0,bobby jindal raised hindu uses story of christ...
3,3,SATAN 2: Russia unvelis an image of its terrif...,1,satan 2 russia unvelis an image of its terrify...
4,4,About Time! Christian Group Sues Amazon and SP...,1,about time christian group sues amazon and spl...
...,...,...,...,...
71420,71420,Russians steal research on Trump in hack of U....,0,russians steal research on trump in hack of u ...
71421,71421,WATCH: Giuliani Demands That Democrats Apolog...,1,watch giuliani demands that democrats apologiz...
71422,71422,Migrants Refuse To Leave Train At Refugee Camp...,0,migrants refuse to leave train at refugee camp...
71423,71423,Trump tussle gives unpopular Mexican leader mu...,0,trump tussle gives unpopular mexican leader mu...


In [18]:
#XY train_test split + tokenize X_train and X_test

X_train, X_test, Y_train, Y_test = train_test_split(df['text_clean'],
                                                    df['label'], 
                                                    test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=df['label'])

X_train_tokens = X_train.apply(get_tokens, args=(tokenizer, 50)) 
X_test_tokens = X_test.apply(get_tokens, args=(tokenizer, 50))

In [19]:
X_train_tokens.shape

(57140,)

In [20]:
Y_train.shape

(57140,)

In [21]:
#creation of 3 tensors: tokens, input masks and target labels

from torch.utils.data import TensorDataset 

input_ids_train = torch.tensor( 
    [features[0] for features in X_train_tokens.values], dtype=torch.long) 
input_mask_train = torch.tensor( 
    [features[1] for features in X_train_tokens.values], dtype=torch.long) 
label_ids_train = torch.tensor(Y_train.values, dtype=torch.long) 

print (input_ids_train.shape) 
print (input_mask_train.shape) 
print (label_ids_train.shape)

torch.Size([57140, 50])
torch.Size([57140, 50])
torch.Size([57140])


In [22]:
input_ids_train[1]

tensor([  101, 29090,  2015, 28896,  1998,  1037,  5656,  2000,  2562,  4268,
         7965, 29090,  2015, 28896,  1998,  1037,  5656,  2000,  2562,  4268,
         7965, 29090,  2015, 28896,  1998,  1037,  5656,  2000,  2562,  4268,
         7965,  1037,  9610, 18981, 22648,  4263,  4391,  1996, 17404,  5379,
         2933,  2011,  8129,  3389,  1040,  1039,  2852,  6673,  6132,   102])

In [23]:
# combine tensors into a tensordataset

train_dataset = TensorDataset(input_ids_train,input_mask_train,label_ids_train)

In [24]:
# training the model

from torch.utils.data import DataLoader, RandomSampler

train_batch_size = 64
num_train_epochs = 2
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset,
sampler=train_sampler,
batch_size=train_batch_size)
t_total = len(train_dataloader) // num_train_epochs
print ("Num examples = ", len(train_dataset))
print ("Num Epochs = ", num_train_epochs)
print ("Total train batch size = ", train_batch_size)
print ("Total optimization steps = ", t_total)

Num examples =  57140
Num Epochs =  2
Total train batch size =  64
Total optimization steps =  446


In [25]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [26]:
learning_rate = 1e-4
adam_epsilon = 1e-8
warmup_steps = 0
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer,
num_warmup_steps=warmup_steps,
num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [27]:
from tqdm.notebook import tqdm

In [28]:
from tqdm import trange, notebook

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_iterator = trange(num_train_epochs, desc="Epoch")

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
print(device)

cuda


In [31]:
torch.cuda.is_available()

True

In [32]:
# Put model in 'train' mode
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [33]:
for epoch in train_iterator:
    epoch_iterator = notebook.tqdm(train_dataloader, desc="Iteration")
    for step, batch in enumerate(epoch_iterator):
      
# Reset all gradients at start of every iteration
        model.zero_grad()
# Put the model and the input observations to GPU
        model.to(device)
        batch = tuple(t.to(device) for t in batch)
# Identify the inputs to the model
        inputs = {'input_ids': batch[0],
                    'attention_mask': batch[1],
                    'labels': batch[2]}
# Forward Pass through the model. Input -> Model -> Output
        outputs = model(**inputs)
# Determine the deviation (loss)
        loss = outputs[0]
        print("\r%f" % loss, end='')
# Back-propogate the loss (automatically calculates gradients)
        loss.backward()
# Prevent exploding gradients by limiting gradients to 1.0
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
# Update the parameters and learning rate
        optimizer.step()
        scheduler.step()

Iteration:   0%|          | 0/893 [00:00<?, ?it/s]

0.146837

Epoch:  50%|█████     | 1/2 [08:30<08:30, 510.61s/it]

Iteration:   0%|          | 0/893 [00:00<?, ?it/s]

0.082007

Epoch: 100%|██████████| 2/2 [16:30<00:00, 495.47s/it]


In [34]:
model.save_pretrained('gdrive/My Drive/Colab Notebooks/FAKE_NEWS/Models/text')

## Test set

In [35]:
# creation of 3 tensors: tokens, input masks and target labels

from torch.utils.data import TensorDataset 

input_ids_test = torch.tensor( 
    [features[0] for features in X_test_tokens.values], dtype=torch.long) 
input_mask_test = torch.tensor( 
    [features[1] for features in X_test_tokens.values], dtype=torch.long) 
label_ids_test = torch.tensor(Y_test.values, dtype=torch.long) 

print (input_ids_test.shape) 
print (input_mask_test.shape) 
print (label_ids_test.shape)

torch.Size([14285, 50])
torch.Size([14285, 50])
torch.Size([14285])


In [36]:
# combine tensors into a tensordataset

test_dataset = TensorDataset(input_ids_test,input_mask_test,label_ids_test)

In [41]:
from torch.utils.data import SequentialSampler

In [42]:
test_batch_size = 64
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset,
                              sampler=test_sampler,
                              batch_size=test_batch_size)

In [43]:
model = model.from_pretrained('gdrive/My Drive/Colab Notebooks/FAKE_NEWS/Models/text')

In [44]:
# Initialize the prediction and actual labels
preds = None
out_label_ids = None

In [45]:
# Put model in "eval" mode
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [47]:
import numpy as np

In [48]:
for batch in notebook.tqdm(test_dataloader, desc="Evaluating"):
  model.to(device)
  batch = tuple(t.to(device) for t in batch)

# Do not track any gradients since in 'eval' mode
  with torch.no_grad():
      inputs = {'input_ids': batch[0],
                 'attention_mask': batch[1],
                  'labels': batch[2]}

# Forward pass through the model
  outputs = model(**inputs)

# We get loss since we provided the labels
  tmp_eval_loss, logits = outputs[:2]

# There may be more than one batch of items in the test dataset
  if preds is None:
    preds = logits.detach().cpu().numpy()
    out_label_ids = inputs['labels'].detach().cpu().numpy()
  else:
    preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
    out_label_ids = np.append(out_label_ids,
    inputs['labels'].detach().cpu().numpy(),
    axis=0)

Evaluating:   0%|          | 0/224 [00:00<?, ?it/s]

In [49]:
preds

array([[-0.5455046,  0.5740679],
       [ 3.232966 , -2.907264 ],
       [-3.270873 ,  3.042159 ],
       ...,
       [-1.9159864,  1.6785899],
       [-1.586339 ,  1.2854159],
       [ 4.1746793, -4.264265 ]], dtype=float32)

In [50]:
# Get final loss, predictions and accuracy

from sklearn.metrics import accuracy_score

preds = np.argmax(preds, axis=1)
acc_score = accuracy_score(preds, out_label_ids)
print ('Accuracy Score on Test data ', acc_score)

Accuracy Score on Test data  0.9688480033451808
